# Modules

In [8]:
import datetime
import json
import os
import sys

# if you are unable to load pdathome.constants, you need to add the path to the src folder to the system path
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

from IPython.display import clear_output

from paradigma.preprocessing_config import IMUPreprocessingConfig

from pdathome.constants import *
from pdathome.load import load_sensor_data, load_video_annotations
from pdathome.preprocessing import determine_wrist_pos, sync_video_annotations, preprocess_video_annotations, \
    preprocess_sensor_data, attach_label_to_signal, determine_med_stage, rotate_axes

# Constants

In [9]:
l_ids = ['hbv002', 'hbv012', 'hbv053']

config = IMUPreprocessingConfig()

# Load data

In [10]:
with open(os.path.join(PATH_CLINICAL_DATA, 'distribution_participants.json'), 'r') as f:
    d_participant_distribution = json.load(f)

# Processing

In [5]:
for subject in [x for x in L_PD_IDS + L_HC_IDS if x not in l_ids]:
    l_time_cols = [config.time_colname]
    l_sensor_cols = L_ACCEL_COLS + L_GYRO_COLS
    l_other_cols = ['free_living_label']

    if subject in L_PD_IDS:
        file_sensor_data = 'sensor_data_pd.mat'
        path_annotations = PATH_ANNOTATIONS_PD
        l_other_cols += ['arm_label', 'pre_or_post']
    else:
        file_sensor_data = 'sensor_data_controls.mat'
        path_annotations = PATH_ANNOTATIONS_CONTROLS

    if subject in L_TREMOR_IDS:
        l_other_cols += ['tremor_label']

    l_cols_to_export = l_time_cols + l_sensor_cols + l_other_cols

    l_dfs = []
    for side in [MOST_AFFECTED_SIDE, LEAST_AFFECTED_SIDE]:

        if os.path.exists(os.path.join(PATH_DATAFRAMES, f'{subject}_{side}.pkl')):
            continue
        
        print(f"Time {datetime.datetime.now()} - {subject} {side} - Loading data ...")

        ## loading_annotations
        wrist_pos = determine_wrist_pos(subject, side, d_participant_distribution)
        df_sensors, peakstart, peakend = load_sensor_data(PATH_SENSOR_DATA, file_sensor_data, 'phys', subject, wrist_pos)

        if subject in L_PD_IDS:
            df_annotations_part_1, df_annotations_part_2 = load_video_annotations(path_annotations, subject)
            df_annotations = sync_video_annotations(df_annotations_part_1, df_annotations_part_2, peakstart, peakend)
        else:
            df_annotations = load_video_annotations(path_annotations, subject)

        df_annotations = preprocess_video_annotations(df_annotations, D_LABELS, D_TIERS_MAP)
        
        ## merging
        df_sensors['side'] = side

        print(f"Time {datetime.datetime.now()} - {subject} {side} - Merging data ...")

        df_sensors = preprocess_sensor_data(df_sensors)

        df_sensors = attach_label_to_signal(df_sensors, df_annotations, 'label', 'tier', 'start_s', 'end_s')

        if subject in L_PD_IDS:
            if wrist_pos == RIGHT_WRIST:
                df_sensors['arm_label'] = df_sensors['right_arm_label']
            else:
                df_sensors['arm_label'] = df_sensors['left_arm_label']

            df_sensors['arm_label'] = df_sensors['arm_label'].fillna('non_gait')

        ## 2.filtering
        print(f"Time {datetime.datetime.now()} - {subject} {side} - Preprocessing data ...")

        df_sensors = determine_med_stage(df=df_sensors, subject=subject,
                                         watch_side=side,
                                         path_annotations=path_annotations)
        df_sensors = rotate_axes(df=df_sensors, subject=subject, wrist=wrist_pos)

        df_sensors = df_sensors.loc[~df_sensors['free_living_label'].isna()]
        df_sensors = df_sensors.loc[df_sensors['free_living_label']!='Unknown']
        df_sensors = df_sensors.reset_index(drop=True)

        l_drop_cols = ['clinical_tests_label']
        if subject in L_PD_IDS:
            l_drop_cols += ['med_and_motor_status_label', 'left_arm_label', 'right_arm_label']

        df_sensors = df_sensors.drop(columns=l_drop_cols)

        # temporarily store as pickle until tsdf issue is resolved
        df_sensors[l_cols_to_export].to_pickle(os.path.join(PATH_DATAFRAMES, f'{subject}_{side}.pkl'))

        clear_output(wait=False)
